<img src="https://www.universidades.com.ec/logos/original/logo-universidad-politecnica-salesiana.png" style="margin: 0 auto"/>

<h1 style="text-align:center;color: darkblue">Desarrollo de un modelo SIR</h1>

<ul style="text-align:center;list-style:none">
    <li><strong>Autor: </strong> Bryam David Vega Moreno</li>
    <li><strong>Maestro: </strong> Diego Quisi</li>
    <li><strong>Materia: </strong> Simulación</li>
    <li><strong>Universidad: </strong> Universidad Politécnica Salesiana</li>
    <li><strong>Carrera: </strong> Ciencias de la computación</li>
</ul>

------------

<h2 style="color:yellowgreen">Introducción</h2>

El modelo SIR es un modelo matemático que permite predecir el comportamiento de una enfermedad infecciosa, a partir de ciertas condiciones iniciales

<img src="https://media.giphy.com/media/MDb57lGm0VFT7IYoTj/giphy.gif" style="margin: 0 auto"/>

Este modelo clasifica una población en tres grupos distintos:
<ul>
    <li><strong>Suceptible: </strong> Número de personas que son propensas a la enfermedad </li>
    <li><strong>Infectado: </strong>  Número de personas que tienen la enfermedad y que pueden infectar a la gente susceptible</li>
    <li><strong>Recuperado: </strong> Númeto de persoans que no pueden contraer la enfermedad, porque se han recuperado completamente, o porque son inmunes  </li>
</ul>

Ademas de estos grupos, este modelo depende principalmente de tres parámetros:
<ul>
    <li><strong>Tasa de transmisión (β): </strong> Describe que tan rápido se transmite la infección de un individuo a otro </li>
    <li><strong>Tasa de recuperación (γ): </strong>  Describe qué tan rápido un individuo infectado se recupera.</li>
    <li><strong>Población (N): </strong>  Número de población del país.</li>
</ul>

Para poder resolver este modelo debemos hacer uso de las siguientes fórmulas:

\begin{align*}
\frac{\mathrm{d}S}{\mathrm{d}t} &= -\beta S \frac{ I}{N},\\
\frac{\mathrm{d}I}{\mathrm{d}t} &= \beta S \frac{ I}{N} - \gamma I,\\
\frac{\mathrm{d}R}{\mathrm{d}t} &= \gamma I.
\end{align*}

Para poder realizar un modelo básico SIR, utilizaremos la libreria SciPy el cual contiene un conjunto de paquetes que permiten resolver sistemas de ecuaciones, integrarlas y graficarlas. A continuación mostramos el código

Ahora, cuando podemos estimar beta y gamma, hay varios conocimientos derivados de el. Si D, es el promedio de días  para recuperarse de infecciones , se deriba de gamma.

\begin{align*}
D= \frac{ I}{\gamma}
\end{align*}

Ademas de ello, podemos estimar la naturaleza de la enfermadad en terminos del poder de la infección.

\begin{align*}
R_0= \frac{ \beta}{\gamma}
\end{align*}

Se llama número de reproducción básico **R0**. Es el número promedio de personas infectadas de otra persona. Si es alto, la probabilidad de pandemia también es mayor. Si bien antes teniamos como variables fijas beta y gamma, ahora lo que procederemos a hacer es estimar β y γ para ajustar el modelo SIR a los casos confirmados reales (el número de personas infecciosas). A continuación vamos a proceder con el código y la explicación

-----------------------------------------

<h2 style="color:yellowgreen">Librerias a importar</h2>

**Para el análisis de datos**

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime

**Librerias para visualización de datos**

In [2]:
import altair as alt
import matplotlib.pyplot as plt

**Librerias para resolución de ecuaciones diferenciales**

In [3]:
import scipy.integrate as spi
from scipy.optimize import minimize
from scipy.integrate import solve_ivp

----------------------------------------------

<h2 style="color:yellowgreen">Simulación del modelo SIR para COVID-19</h2>

Lo primero que procedemos a hacer es crear nuestras variables I0,R0,S0 con la finaliad de tener datos iniciales, estos datos pueden ir variando para ir simulando los diferentes escenarios. Para los valores tomados, tendremos en cuenta que existiran 10 millones suceptibles, mientras que habran 10 infectados y 0 recuperados. De la misma manera tomaremos 365 días a partir del día del primer caso. A continuación mostramos las variables

In [4]:
I0=10
R0=0
S0 = 100000
t = 365
y0 = S0,I0,R0

In [5]:
path_confirmed=''
path_recovered=''
country=''
date=''

Con las variables y condiciones iniciales procedemos a leer los datos que necesitamos para poder realizar nuestro modelo SIR, en este caso utilizamos los dataset de numero de confirmados y numero de recuperados con la finalidad de que se pueda obtener una mejor curva para los recovered puesto que después de un conjunto de experimentos, existia un problema en el número de recuperados. El código de lectura y obtención de datos es el siguiente:

In [6]:
def filter_country(df,country,start_date):
    country_df = df[df['Country/Region'] == country]
    return country_df.iloc[0].loc[start_date:]

def load_data(path_confirmed,path_recovered,country,date):
    df_confirmed = filter_country(pd.read_csv(path_confirmed),country,date)
    df_recovered = filter_country(pd.read_csv(path_recovered),country,date)
    return df_confirmed,df_recovered

In [ ]:
data_confirmed,data_recovered=load_data(path_confirmed,path_recovered,country,date)

Ahora procedemos a realizar una función con el fin de optimizar los valores de beta y gamma con el fin de mejorar dichos valores con respecto a la data que tenemos, para ello utilizamos dos funciones para a la final ver sus diferencias, en la primera función **loss_confirmed_recovered** utilizamos la data de personas infectadas y recuperadas a fin de mejorar la curva de recovered, mientras que en la otra función **loss_confirmed** solamente utilizamos la data de infectados. A continuación mostramos la función que nos permite optimizar dicho proceso. 

**NOTA**

A partir de este punto vamos a optener dos soluciones, debido a que tenemos diferentes funciones de optimización y tomamos en cuenta diferentes combinaciones de tipos de datos, con esto en cuenta vamos a referirnos a dichas funciones con la siguiente nomenclatura a patir de aqui:
* **loss_confirmed_recovered:** primera función
* **loss_confirmed:** segunda función

In [7]:
def loss_confirmed_recovered(point, data, recovered):
    size = len(data)
    beta, gamma = point
    def SIR(t, y):
        S = y[0]
        I = y[1]
        R = y[2]
        return [-beta*S*I, beta*S*I-gamma*I, gamma*I]
    solution = solve_ivp(SIR, [0, size], [S0,I0,R0], t_eval=np.arange(0, size, 1), vectorized=True)
    l1 = np.sqrt(np.mean((solution.y[1] - data)**2))
    l2 = np.sqrt(np.mean((solution.y[2] - recovered)**2))
    alpha = 0.1
    return alpha * l1 + (1 - alpha) * l2

def loss_confirmed(point, data):
    size = len(data)
    beta, gamma = point
    def SIR(t, y):
        S = y[0]
        I = y[1]
        R = y[2]
        return [-beta*S*I, beta*S*I-gamma*I, gamma*I]
    solution = solve_ivp(SIR, [0, size], [S0,I0,R0], t_eval=np.arange(0, size, 1), vectorized=True)
    return np.sqrt(np.mean((solution.y[1] - data)**2))

En el siguiente código procedemos a realizar la optimización para ambos casos explicados anteriormente

In [ ]:
optimal_cr = minimize(
            loss_confirmed_recovered,
            [0.001, 0.001],
            args=(data_confirmed,data_recovered),
            method='L-BFGS-B',
            bounds=[(0.00000001, 0.4), (0.00000001, 0.4)])

optimal_c = minimize(
            loss_confirmed,
            [0.001, 0.001],
            args=(data_confirmed),
            method='L-BFGS-B',
            bounds=[(0.00000001, 0.4), (0.00000001, 0.4)])

In [ ]:
beta_cr,gamma_cr = optimal_cr.x
beta_c,gamma_c = optimal_c.x

Despues de optimizar dichos valores, procedemos a motrar los valores que obtuvimos con la primera función

In [ ]:
print("valor gamma confirmed_recovered: {}".format(gamma_cr))
print("valor beta confirmed_recovered: {}".format(beta_cr))

Mientras que por otro lado, estos son los valores que obtuvimos con la segunda función

In [ ]:
print("valor gamma confirmed: {}".format(gamma_c))
print("valor beta confirmed: {}".format(beta_c))

Con dichos valores procedemos a obtener R_0 con el objetivo de conocer mas que nada su valor. En este caso mostrarmos el R_0 con los valores de beta y gamma de la primera función

In [ ]:
R_0= beta_cr/gamma_cr
print("Número de reproducción R_0 confirmed_recovered: {}".format(R_0))

Mientras que por otro lado mostramos el valor de R_0 con los valores de beta y gamma de la segunda función

In [ ]:
R_0= beta_c/gamma_c
print("Número de reproducción R_0 confirmed: {}".format(R_0))

Una vez obtenido el valor de R_0 el cual es un indicador para ver el número de reproducción del virus, procedemos a realizar la resolución del modelo SIR, no necesitamos de R_0 en este momento, simplemente es un indicador que nos permite saber como se reproduce el virus en nuestra ciudad. Ahora con ello en mente mostramos el código en donde resolvemos el modelo SIR

In [8]:
def extend_index(index, new_size):
        values = index.values
        current = datetime.strptime(index[-1], '%m/%d/%y')
        while len(values) < new_size:
            current = current + timedelta(days=1)
            values = np.append(values, datetime.strftime(current, '%m/%d/%y'))
        return values

def predict(beta, gamma, data):
        predict_range = t
        new_index = extend_index(data.index, predict_range)
        size = len(new_index)
        def SIR(t, y):
            S = y[0]
            I = y[1]
            R = y[2]
            return [-beta*S*I, beta*S*I-gamma*I, gamma*I]
        extended_actual = np.concatenate((data.values, [None] * (size - len(data.values))))
        return new_index, extended_actual, solve_ivp(SIR, [0, size], [S0,I0,R0], t_eval=np.arange(0, size, 1))

Procedemos a realizar las predicciones para nuestros datos, teniendo en cuenta el beta y gamma. A continuación realizamos dichas predicciones para la primera y segunda función

In [ ]:
new_index, extended_actual, prediction_cr = predict(beta_cr, gamma_cr, data_confirmed)
new_index, extended_actual, prediction_c = predict(beta_c, gamma_c, data_confirmed)

Después de realizar las predicciones para el modelo SIR procedemos a armar un dataframe para poder visualizar nuestro modelo terminado. A la final tendremos dos datasets ya que el uno es de la primera función, mientras que el otro es de la segunda función

In [ ]:
df_cr = pd.DataFrame(
            {'date':[i for i in range(0,len(new_index))],
            'suceptible': prediction_cr.y[0],
            'infected': prediction_cr.y[1],
            'recovered': prediction_cr.y[2]})
df_c = pd.DataFrame(
            {'date':[i for i in range(0,len(new_index))],
            'suceptible': prediction_c.y[0],
            'infected': prediction_c.y[1],
            'recovered': prediction_c.y[2]})

#### Dataset Confirmed recovered Cases

In [ ]:
df_cr.head(5)

In [ ]:
alt.Chart(df_cr.melt('date')).mark_line().encode(
    x='date',
    y=alt.Y('value'),
    color='variable'
).properties(
    title='SIR model for Ecuador with confirmed and recovered cases'
).interactive()

#### Dataset confirmed Cases

In [ ]:
df_c.head(5)

In [ ]:
alt.Chart(df_c.melt('date')).mark_line().encode(
    x='date',
    y=alt.Y('value'),
    color='variable'
).properties(
    title='SIR model for Ecuador only confirmed cases'
).interactive()

Como podemos apreciar, con el uso de la primera función existe una mejor visualización de las personas recuperadas, es decir, es muy importante tomar en cuenta dicho valor para que exista una curva correcta con respecto a los casos recuperados. Sin embargo, como podemos ver en la gráfica de la segunda función en donde solamente se tomo en cuenta el número de personas confrmadas, podemos notar claramente como no existe número de personas récuperadas y al no existir recuperadas por ende dichas personas van a seguir infectadas, por lo cual esto es erroneo ya que si existen personas recuperadas. Por lo tanto la mejor manera de simular el modelo SIR, es con la función de optimización **loss_confirmed_recovered** ya que toma en cuenta a las personas infectadas y recuperadas y de esa manera podemos ver que al existir recuperados, el número de infectados bajan lo cual tiene lógica.